In [ ]:
%load_ext autoreload
%autoreload 2

import os
import time
import glob
import gdown
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import csv
import cv2
import itertools
import matplotlib.pyplot as plt
import skimage.feature as feature
import xlwings as xw
import torchvision.transforms as transforms

import random

#libraries for yolo
from pytorchyolo.models import load_model
from pytorchyolo.utils.transforms import Resize, DEFAULT_TRANSFORMS
from pytorchyolo.utils.utils import non_max_suppression
from pytorchyolo.utils.loss import compute_loss

from matplotlib.ticker import (FormatStrFormatter, AutoMinorLocator, FuncFormatter, )

In [ ]:
def download_weights():
    model_file=[
        'yolo_face_sthanhng.weights',
        'yolo_face_sthanhng.cfg'
    ]
    
    gdrive_url=[
        'https://drive.google.com/uc?id=1utquM5TAnfIa1Aq0X9fCvrllHiTWazdD',
        'https://drive.google.com/uc?id=1CPUZlYL5ik4d9y6oCyzi0930KgzawI6V'
    ]
    
    cwd=os.getcwd() 
    if 'weights' in os.listdir(cwd):
        for i in range(len(model_file)):
            if model_file[i] in os.listdir(os.path.join(cwd, 'weights')):
                print(model_file[i] + ':: status : file already exists')
            else:
                gdown.download(gdrive_url[i],os.path.join(cwd, 'weights', model_file[i]), quiet=False)
    else:
        os.makedirs(os.path.join(cwd,'weights'))
        for i in range(len(model_file)):
            gdown.download(gdrive_url[i], os.path.join(cwd, 'weights', model_file[i]), quiet=False)  

In [ ]:
# download the necessary weights for YOLO-Face
download_weights()

## YOLOFace with FGSM

In [ ]:
# Patterned after FGSM tutorial (https://pytorch.org/tutorials/beginner/fgsm_tutorial.html)
# Define what device we are using
print("CUDA Available: ", torch.cuda.is_available())
device, model = load_model('./weights/yolo_face_sthanhng.cfg', "./weights/yolo_face_sthanhng.weights")

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

epsilons = [0, .05]
use_cuda=True

In [ ]:
"""
# FGSM attack code
def fgsm_attack(image, epsilon, data_grad, x1, y1, x2, y2):
    # Collect the element-wise sign of the data gradient
    image = image
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image
    perturbed_image[:, :, y1:y2, x1:x2] = perturbed_image[:, :, y1:y2, x1:x2] + epsilon * sign_data_grad[:, :, y1:y2, x1:x2] # apply it only to the face region
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image
"""

## Image Feature Extraction

In [ ]:
class LocalBinaryPatterns:
  def __init__(self, numPoints, radius):
    self.numPoints = numPoints
    self.radius = radius

  def describe(self, image, eps = 1e-7):
    lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.numPoints+3), range=(0, self.numPoints + 2))

    # Normalize the histogram
    hist = hist.astype('float')
    hist /= (hist.sum() + eps)

    return hist, lbp

# From https://medium.com/mlearning-ai/how-to-plot-color-channels-histogram-of-an-image-in-python-using-opencv-40022032e127
# Extracts image's color channel
def extract_color_channel(path, face_index, image):
    # BGR Image Color Conversion
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    COLOR_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_Colors')
    if not os.path.exists(COLOR_PATH):
        os.mkdir(COLOR_PATH)
    
    cv2.imwrite(os.path.join(COLOR_PATH, os.path.splitext(os.path.basename(path))[0]) + '_RGB_' + str(face_index) + '.png', rgb)
    cv2.imwrite(os.path.join(COLOR_PATH, os.path.splitext(os.path.basename(path))[0]) + '_HSV_' + str(face_index) + '.png', hsv)
    cv2.imwrite(os.path.join(COLOR_PATH, os.path.splitext(os.path.basename(path))[0]) + '_HSL_' + str(face_index) + '.png', hls)
    cv2.imwrite(os.path.join(COLOR_PATH, os.path.splitext(os.path.basename(path))[0]) + '_LAB_' + str(face_index) + '.png', lab)
    cv2.imwrite(os.path.join(COLOR_PATH, os.path.splitext(os.path.basename(path))[0]) + '_YCRCB_' + str(face_index) + '.png', ycrcb)

#     # RGB Image Histogram
#     red_hist = cv2.calcHist([rgb], [0], None, [256], [0, 256])
#     green_hist = cv2.calcHist([rgb], [1], None, [256], [0, 256])
#     blue_hist = cv2.calcHist([rgb], [2], None, [256], [0, 256])

#     # HSV Image Histogram
#     hue_hist_HSV = cv2.calcHist([hsv], [0], None, [256], [0, 256])
#     saturation_hist_HSV = cv2.calcHist([hsv], [1], None, [256], [0, 256])
#     value_hist = cv2.calcHist([hsv], [2], None, [256], [0, 256])

#     # HLS Image Histogram
#     hue_hist_HLS = cv2.calcHist([hls], [0], None, [256], [0, 256])
#     lightness_hist_HLS = cv2.calcHist([hls], [1], None, [256], [0, 256])
#     saturation_hist_HLS = cv2.calcHist([hls], [2], None, [256], [0, 256])

#     # LAB Image Histogram
#     lightness_hist_LAB = cv2.calcHist([lab], [0], None, [256], [0, 256])
#     a_hist_LAB = cv2.calcHist([lab], [1], None, [256], [0, 256])
#     b_hist_LAB = cv2.calcHist([lab], [2], None, [256], [0, 256])

#     # YCrCb Image Histogram
#     y_hist = cv2.calcHist([ycrcb], [0], None, [256], [0, 256])
#     cr_hist = cv2.calcHist([ycrcb], [1], None, [256], [0, 256])
#     cb_hist = cv2.calcHist([ycrcb], [2], None, [256], [0, 256])

#     # RGB Image Plot
#     plt.subplot(4, 1, 1)
#     plt.imshow(rgb)
#     plt.title('RGB Image')
#     plt.xticks([])
#     plt.yticks([])

#     plt.subplot(4, 1, 2)
#     plt.plot(red_hist, color='r')
#     plt.xlim([0, 256])
#     plt.ylim([0, 500])
#     plt.title('Red Histogram')

#     plt.subplot(4, 1, 3)
#     plt.plot(green_hist, color='g')
#     plt.xlim([0, 256])
#     plt.ylim([0, 500])
#     plt.title('Green Histogram')

#     plt.subplot(4, 1, 4)
#     plt.plot(blue_hist, color='b')
#     plt.xlim([0, 256])
#     plt.ylim([0, 500])
#     plt.title('Blue Histogram')

#     plt.tight_layout()
#     #plt.show()

    r, g, b = cv2.split(rgb)
    
    r *= 255
    g *= 255
    b *= 255
    
    RGB_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_RGB')
    if not os.path.exists(RGB_PATH):
        os.mkdir(RGB_PATH)
    
    cv2.imwrite(os.path.join(RGB_PATH, os.path.splitext(os.path.basename(path))[0]) + '_R_RGB_' + str(face_index) + '.png', r)
    cv2.imwrite(os.path.join(RGB_PATH, os.path.splitext(os.path.basename(path))[0]) + '_G_RGB_' + str(face_index) + '.png', g)
    cv2.imwrite(os.path.join(RGB_PATH, os.path.splitext(os.path.basename(path))[0]) + '_B_RGB_' + str(face_index) + '.png', b)
    
    r_hist = cv2.calcHist(r, [0], None, [26], [0, 256])
    r_hist = r_hist.ravel()
    r_hist = r_hist.astype('float')
    r_hist /= r_hist.sum()
    
    g_hist = cv2.calcHist([g], [0], None, [26], [0, 256])
    g_hist = g_hist.ravel()
    g_hist = g_hist.astype('float')
    g_hist /= g_hist.sum()
    
    b_hist = cv2.calcHist([b], [0], None, [26], [0, 256])
    b_hist = b_hist.ravel()
    b_hist = b_hist.astype('float')
    b_hist /= b_hist.sum()

#     # HSV Image Plot
#     plt.subplot(4, 1, 1)
#     #plt.imshow(hsv)
#     plt.title('HSV Image')
#     plt.xticks([])
#     plt.yticks([])

#     plt.subplot(4, 1, 2)
#     plt.plot(hue_hist_HSV, color='c')
#     plt.xlim([0, 256])
#     plt.ylim([0, 2500])
#     plt.title('Hue Histogram')

#     plt.subplot(4, 1, 3)
#     plt.plot(saturation_hist_HSV, color='m')
#     plt.xlim([0, 256])
#     plt.ylim([0, 1000])
#     plt.title('Saturation Histogram')

#     plt.subplot(4, 1, 4)
#     plt.plot(value_hist, color='y')
#     plt.xlim([0, 256])
#     plt.ylim([0, 1000])
#     plt.title('Value Histogram')

#     plt.tight_layout()
#     plt.show()
    
    h, s, v = cv2.split(hsv)
    
    s *= 255
    v *= 255
    
    HSV_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_HSV')
    if not os.path.exists(HSV_PATH):
        os.mkdir(HSV_PATH)
    
    cv2.imwrite(os.path.join(HSV_PATH, os.path.splitext(os.path.basename(path))[0]) + '_H_HSV_' + str(face_index) + '.png', h)
    cv2.imwrite(os.path.join(HSV_PATH, os.path.splitext(os.path.basename(path))[0]) + '_S_HSV_' + str(face_index) + '.png', s)
    cv2.imwrite(os.path.join(HSV_PATH, os.path.splitext(os.path.basename(path))[0]) + '_V_HSV_' + str(face_index) + '.png', v)
    
    h_hist_HSV = cv2.calcHist([h], [0], None, [36], [0, 361])
    h_hist_HSV = h_hist_HSV.ravel()
    h_hist_HSV = h_hist_HSV.astype('float')
    h_hist_HSV /= h_hist_HSV.sum()
    
    s_hist_HSV = cv2.calcHist([s], [0], None, [26], [0, 256])
    s_hist_HSV = s_hist_HSV.ravel()
    s_hist_HSV = s_hist_HSV.astype('float')
    s_hist_HSV /= s_hist_HSV.sum()
    
    v_hist_HSV = cv2.calcHist([v], [0], None, [26], [0, 256])
    v_hist_HSV = v_hist_HSV.ravel()
    v_hist_HSV = v_hist_HSV.astype('float')
    v_hist_HSV /= v_hist_HSV.sum()
    
#     # HLS Image Plot
#     plt.subplot(4, 1, 1)
#     plt.imshow(hls)
#     plt.title('HLS Image')
#     plt.xticks([])
#     plt.yticks([])

#     plt.subplot(4, 1, 2)
#     plt.plot(hue_hist_HLS, color='r')
#     plt.xlim([0, 256])
#     plt.ylim([0, 2500])
#     plt.title('Hue Histogram')

#     plt.subplot(4, 1, 3)
#     plt.plot(lightness_hist_HLS, color='g')
#     plt.xlim([0, 256])
#     plt.ylim([0, 1000])
#     plt.title('Lightness Histogram')

#     plt.subplot(4, 1, 4)
#     plt.plot(saturation_hist_HLS, color='b')
#     plt.xlim([0, 256])
#     plt.ylim([0, 1000])
#     plt.title('Saturation Histogram')

#     plt.tight_layout()
#     plt.show()

    h, l, s = cv2.split(hls)
    
    l *= 255
    s *= 255
    
    HSL_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_HSL')
    if not os.path.exists(HSL_PATH):
        os.mkdir(HSL_PATH)
    
    cv2.imwrite(os.path.join(HSL_PATH, os.path.splitext(os.path.basename(path))[0]) + '_H_HSL_' + str(face_index) + '.png', h)
    cv2.imwrite(os.path.join(HSL_PATH, os.path.splitext(os.path.basename(path))[0]) + '_S_HSL_' + str(face_index) + '.png', s)
    cv2.imwrite(os.path.join(HSL_PATH, os.path.splitext(os.path.basename(path))[0]) + '_L_HSL_' + str(face_index) + '.png', l)
    
    h_hist_HSL = cv2.calcHist([h], [0], None, [36], [0, 361])
    h_hist_HSL = h_hist_HSL.ravel()
    h_hist_HSL = h_hist_HSL.astype('float')
    h_hist_HSL /= h_hist_HSL.sum()
    
    l_hist_HSL = cv2.calcHist([l], [0], None, [26], [0, 256])
    l_hist_HSL = l_hist_HSL.ravel()
    l_hist_HSL = l_hist_HSL.astype('float')
    l_hist_HSL /= l_hist_HSL.sum()
    
    s_hist_HSL = cv2.calcHist([s], [0], None, [26], [0, 256])
    s_hist_HSL = s_hist_HSL.ravel()
    s_hist_HSL = s_hist_HSL.astype('float')
    s_hist_HSL /= s_hist_HSL.sum()
    
#     # LAB Image Plot
#     plt.subplot(4, 1, 1)
#     plt.imshow(lab)
#     plt.title('LAB Image')
#     plt.xticks([])
#     plt.yticks([])

#     plt.subplot(4, 1, 2)
#     plt.plot(lightness_hist_LAB, color='c')
#     plt.xlim([0, 256])
#     plt.ylim([0, 1000])
#     plt.title('Lightness Histogram')

#     plt.subplot(4, 1, 3)
#     plt.plot(a_hist_LAB, color='m')
#     plt.xlim([0, 256])
#     plt.ylim([0, 20000])
#     plt.title('A Histogram')

#     plt.subplot(4, 1, 4)
#     plt.plot(b_hist_LAB, color='y')
#     plt.xlim([0, 256])
#     plt.ylim([0, 20000])
#     plt.title('B Histogram')

#     plt.tight_layout()
#     plt.show()
    
    l, a, b = cv2.split(lab)
    
    LAB_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_LAB')
    if not os.path.exists(LAB_PATH):
        os.mkdir(LAB_PATH)
    
    cv2.imwrite(os.path.join(LAB_PATH, os.path.splitext(os.path.basename(path))[0]) + '_L_LAB_' + str(face_index) + '.png', l)
    cv2.imwrite(os.path.join(LAB_PATH, os.path.splitext(os.path.basename(path))[0]) + '_A_LAB_' + str(face_index) + '.png', a)
    cv2.imwrite(os.path.join(LAB_PATH, os.path.splitext(os.path.basename(path))[0]) + '_B_LAB_' + str(face_index) + '.png', b)
    
    l_hist_LAB = cv2.calcHist([l], [0], None, [26], [0, 256])
    l_hist_LAB = l_hist_LAB.ravel()
    l_hist_LAB = l_hist_LAB.astype('float')
    l_hist_LAB /= l_hist_LAB.sum()
    
    a_hist_LAB = cv2.calcHist([a], [0], None, [26], [0, 256])
    a_hist_LAB = a_hist_LAB.ravel()
    a_hist_LAB = a_hist_LAB.astype('float')
    a_hist_LAB /= a_hist_LAB.sum()
    
    b_hist_LAB = cv2.calcHist([b], [0], None, [26], [0, 256])
    b_hist_LAB = b_hist_LAB.ravel()
    b_hist_LAB = b_hist_LAB.astype('float')
    b_hist_LAB /= b_hist_LAB.sum()
    
#     # YCrCb Image Plot
#     plt.subplot(4, 1, 1)
#     plt.imshow(ycrcb)
#     plt.title('YCrCb Image')
#     plt.xticks([])
#     plt.yticks([])

#     plt.subplot(4, 1, 2)
#     plt.plot(y_hist, color='r')
#     plt.xlim([0, 256])
#     plt.ylim([0, 1000])
#     plt.title('Y Histogram')

#     plt.subplot(4, 1, 3)
#     plt.plot(cr_hist, color='g')
#     plt.xlim([0, 256])
#     plt.ylim([0, 20000])
#     plt.title('Cr Histogram')

#     plt.subplot(4, 1, 4)
#     plt.plot(cb_hist, color='b')
#     plt.xlim([0, 256])
#     plt.ylim([0, 20000])
#     plt.title('Cb Histogram')

#     plt.tight_layout()
#     plt.show()
    
    y, cr, cb = cv2.split(ycrcb)
    
    y *= 255
    cr *= 255
    cb *= 255
    
    YCRCB_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_YCRCB')
    if not os.path.exists(YCRCB_PATH):
        os.mkdir(YCRCB_PATH)
    
    cv2.imwrite(os.path.join(YCRCB_PATH, os.path.splitext(os.path.basename(path))[0]) + '_Y_YCRCB_' + str(face_index) + '.png', y)
    cv2.imwrite(os.path.join(YCRCB_PATH, os.path.splitext(os.path.basename(path))[0]) + '_CR_YCRCB_' + str(face_index) + '.png', cr)
    cv2.imwrite(os.path.join(YCRCB_PATH, os.path.splitext(os.path.basename(path))[0]) + '_CB_YCRCB_' + str(face_index) + '.png', cb)
    
    
    y_hist = cv2.calcHist([y], [0], None, [26], [0, 256])
    y_hist = y_hist.ravel()
    y_hist = y_hist.astype('float')
    y_hist /= y_hist.sum()
    
    cr_hist = cv2.calcHist([cr], [0], None, [26], [0, 256])
    cr_hist = cr_hist.ravel()
    cr_hist = cr_hist.astype('float')
    cr_hist /= cr_hist.sum()
    
    cb_hist = cv2.calcHist([cb], [0], None, [26], [0, 256])
    cb_hist = cb_hist.ravel()
    cb_hist = cb_hist.astype('float')
    cb_hist /= cb_hist.sum()
    
    face_index = str(face_index)
    # rows = itertools.zip_longest([path], [face_index], r_hist, g_hist, b_hist, h_hist_HSV, s_hist_HSV, v_hist_HSV, h_hist_HSL, s_hist_HSL, l_hist_HSL, l_hist_LAB, a_hist_LAB, b_hist_LAB, y_hist, cr_hist, cb_hist)
    
    # with open("color.csv", "a", newline = "") as f:
    #     if os.stat("color.csv").st_size == 0:
    #         csv.writer(f).writerow(["Path", "Face Index", "Red", "Green", "Blue", "Hue_HSV", "Saturation_HSV", "Value_HSV", "Hue_HSL", "Saturation_HSL", "Lightness_HSL", "Lightness_LAB", "A_LAB", "B_LAB", "Y", "Cr", "Cb"])
    #     csv.writer(f).writerows(rows)
    
    return r_hist, g_hist, b_hist, h_hist_HSV, s_hist_HSV, v_hist_HSV, h_hist_HSL, s_hist_HSL, l_hist_HSL, l_hist_LAB, a_hist_LAB, b_hist_LAB, y_hist, cr_hist, cb_hist

# From https://medium.com/mlearning-ai/color-shape-and-texture-feature-extraction-using-opencv-cb1feb2dbd73
# Extracts Local Binary Pattern (Texture) of an image
def extract_lbp(path, face_index, image):
    # reads the input image as a grayscale image
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    desc = LocalBinaryPatterns(24, 8)
    lbp_hist, lbp_img = desc.describe(gray)
    
    LBP_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_LBP')
    if not os.path.exists(LBP_PATH):
        os.mkdir(LBP_PATH)
    
    cv2.imwrite(os.path.join(LBP_PATH, os.path.splitext(os.path.basename(path))[0]) + '_LBP_' + str(face_index) + '.png', lbp_img)
    
    # plt.imshow(lbp_img, cmap = plt.get_cmap('gray'))
    # plt.show()
    # lbp_hist = cv2.calcHist([lbp_img], [0], None, [256], [0, 256])
    
    # face_index = str(face_index)
    # rows = itertools.zip_longest([path], [face_index], lbp_hist)
    
    # with open("lbp.csv", "a", newline = "") as f:
    #     if os.stat("lbp.csv").st_size == 0:
    #         csv.writer(f).writerow(["Path", "Face Index", "LBP"])
    #     csv.writer(f).writerows(rows)
    
    return lbp_hist
    
# From https://docs.opencv.org/4.x/d2/d2c/tutorial_sobel_derivatives.html and https://gist.github.com/rahit/c078cabc0a48f2570028bff397a9e154
def extract_gradients(path, face_index, image):
    # Uses the Sobel Filter to extract the gradients of an image
    # reads the input image, then converts BGR color space to RGB
    # img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # compute the 1st order Sobel derivative in X-direction
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)

    # compute the 1st order Sobel derivative in Y-direction
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
    
    # combine sobelx and sobely to form sobel
    sobel = sobelx + sobely
    
    SOBEL_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_SOBEL')
    if not os.path.exists(SOBEL_PATH):
        os.mkdir(SOBEL_PATH)
    
    cv2.imwrite(os.path.join(SOBEL_PATH, os.path.splitext(os.path.basename(path))[0]) + '_SOBELX_' + str(face_index) + '.png', sobelx)
    cv2.imwrite(os.path.join(SOBEL_PATH, os.path.splitext(os.path.basename(path))[0]) + '_SOBELY_' + str(face_index) + '.png', sobely)
    cv2.imwrite(os.path.join(SOBEL_PATH, os.path.splitext(os.path.basename(path))[0]) + '_SOBEL_' + str(face_index) + '.png', sobel)

    # # display sobelx, sobely, and sobel
    # plt.imshow(sobelx, cmap = "gray")
    # plt.show()
    # plt.imshow(sobely, cmap = "gray")
    # plt.show()
    # plt.imshow(sobel, cmap = "gray")
    # plt.show()
    
    sobelx_hist = cv2.calcHist([np.float32(sobelx)], [0], None, [26], [0, 256])
    sobelx_hist = sobelx_hist.ravel()
    sobelx_hist = sobelx_hist.astype('float')
    sobelx_hist /= sobelx_hist.sum()
    
    sobely_hist = cv2.calcHist([np.float32(sobely)], [0], None, [26], [0, 256])
    sobely_hist = sobely_hist.ravel()
    sobely_hist = sobely_hist.astype('float')
    sobely_hist /= sobely_hist.sum()
    
    sobel_hist = cv2.calcHist([np.float32(sobel)], [0], None, [26], [0, 256])
    sobel_hist = sobel_hist.ravel()
    sobel_hist = sobel_hist.astype('float')
    sobel_hist /= sobel_hist.sum()
    
    # face_index = str(face_index)
    # rows = itertools.zip_longest([path], [face_index], sobelx_hist, sobely_hist, sobel_hist)
    
    # with open("gradient.csv", "a", newline = "") as f:
    #     if os.stat("gradient.csv").st_size == 0:
    #         csv.writer(f).writerow(["Path", "Face Index", "Sobel X", "Sobel Y", "Sobel"])
    #     csv.writer(f).writerows(rows)
    
    return sobelx_hist, sobely_hist, sobel_hist

def extract_image_attributes(row, path, face_index, image):
    r_hist, g_hist, b_hist, h_hist_HSV, s_hist_HSV, v_hist_HSV, h_hist_HSL, s_hist_HSL, l_hist_HSL, l_hist_LAB, a_hist_LAB, b_hist_LAB, y_hist, cr_hist, cb_hist = extract_color_channel(path, face_index, image)
    lbp_hist = extract_lbp(path, face_index, image)
    sobelx_hist, sobely_hist, sobel_hist = extract_gradients(path, face_index, image)
    
    RGB_size = SV_HSV_size = SL_HSL_size = LAB_size = YCRCB_size = 26
    
    for i in range(0, RGB_size):
        row['R_BIN_' + str(i)] = r_hist[i]
        
    for i in range(0, RGB_size):
        row['G_BIN_' + str(i)] = g_hist[i]
        
    for i in range(0, RGB_size):
        row['B_BIN_' + str(i)] = b_hist[i]
        
    for i in range(0, h_hist_HSV.size):
        row['H_HSV_BIN' + str(i)] = h_hist_HSV[i]
        
    for i in range(0, SV_HSV_size):
        row['S_HSV_BIN' + str(i)] = s_hist_HSV[i]
        
    for i in range(0, SV_HSV_size):
        row['V_HSV_BIN' + str(i)] = v_hist_HSV[i]
        
    for i in range(0, h_hist_HSL.size):
        row['H_HSL_BIN' + str(i)] = h_hist_HSL[i]
        
    for i in range(0, SL_HSL_size):
        row['S_HSL_BIN' + str(i)] = s_hist_HSL[i]
        
    for i in range(0, SL_HSL_size):
        row['L_HSL_BIN' + str(i)] = l_hist_HSL[i]
        
    for i in range(0, LAB_size):
        row['L_LAB_BIN' + str(i)] = l_hist_LAB[i]
    
    for i in range(0, LAB_size):
        row['A_LAB_BIN' + str(i)] = a_hist_LAB[i]
        
    for i in range(0, LAB_size):
        row['B_LAB_BIN' + str(i)] = b_hist_LAB[i]
        
    for i in range(0, YCRCB_size):
        row['Y_BIN' + str(i)] = y_hist[i]
        
    for i in range(0, YCRCB_size):
        row['CR_BIN' + str(i)] = cr_hist[i]
        
    for i in range(0, YCRCB_size):
        row['CB_BIN' + str(i)] = cb_hist[i]
        
    for i in range(0, lbp_hist.size):
        row["LBP_BIN" + str(i)] = lbp_hist[i]
        
    for i in range(0, sobelx_hist.size):
        row["SOBELX_BIN" + str(i)] = sobelx_hist[i]
        
    for i in range(0, sobely_hist.size):
        row["SOBELY_BIN" + str(i)] = sobely_hist[i]
        
    for i in range(0, sobel_hist.size):
        row["SOBEL_BIN" + str(i)] = sobel_hist[i]
    
    return row

In [ ]:
def detach_cpu(image):
    return image.detach().cpu()

# convert 1x3x416x416 to 416x416x3
def reshape_image(image):
    return np.transpose(np.squeeze(image), (1 ,2, 0))

# convert 1x3x416x416 tensor to 416x416x3 numpy image
def tensor_to_image(image):
    return np.transpose(image.detach().cpu().squeeze().numpy(), (1, 2, 0))

def save_tensor_as_image(image, path):
    save_img = cv2.cvtColor(np.moveaxis((image.detach().numpy() * 255).squeeze(), 0, -1).astype('uint8'), cv2.COLOR_RGB2BGR)
    cv2.imwrite(path, save_img)

In [ ]:
import minepsilon as minE

In [ ]:
def load_mask(filename, face_num, target_bbox):
    faces_df = pd.read_csv(CSV_FILE)
    filename = "restored_mask_" + os.path.splitext(filename)[0] + "_" + str(face_num) + "_image_final.png"
    mask = cv2.imread(os.path.join(os.getcwd(), RESTORED_MASK_PATH, filename), 0)
    
    face_row = faces_df.loc[faces_df['filename'] == filename]
    padded_dim = (int(face_row["x2_pad"] - face_row["x1_pad"]), int(face_row["y2_pad"] - face_row["y1_pad"]))
    target_dim = (int(target_bbox[2] - target_bbox[0]), int(target_bbox[3] - target_bbox[1]))
    
    if dict(zip(*np.unique(mask, return_counts = True)))[255] < int(target_dim[0] * target_dim[1] * 0.5):
        return torch.ones((1, 3, target_dim[1], target_dim[0])), False
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (int(mask.shape[0] * 0.5), int(mask.shape[1] * 0.5)))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    mask = transforms.Compose([DEFAULT_TRANSFORMS])((mask, np.zeros((1, 5))))[0].unsqueeze(0)
    
    #print(os.path.join(os.getcwd(), RESTORED_MASK_PATH, filename))
    #print("mask size before:", mask.shape)
    
    
    #print("x1, y1, orig shape")
    #print(int(face_row["x1_pad"]), int(face_row["y1_pad"]), int(face_row["x2_pad"]), int(face_row["y2_pad"]))
    #print(original_shape)
    
    #print("target shape:", original_shape)
    #print("yoloshape:", int(face_row["x2"] - face_row["x1"]), int(face_row["y2"] - face_row["y1"]))
    
    current_dim = max(mask.shape)
    diff_x, diff_y = abs(padded_dim[0] - current_dim) / 2, abs(padded_dim[1] - current_dim) / 2
    #print("first diff:", diff_x, diff_y)
    
    if diff_y != 0:
        mask = mask[..., int(np.floor(diff_y)):-int(np.ceil(diff_y)), :]
    if diff_x != 0:
        mask = mask[..., int(np.floor(diff_x)):-int(np.ceil(diff_x))]
        
    #print(mask.shape == padded_dim, mask.shape, padded_dim, target_dim)
    
    padding = [
        int(abs(face_row["x1"] - face_row["x1_pad"])),
        int(abs(face_row["y1"] - face_row["y1_pad"])),
        int(abs(face_row["x2"] - face_row["x2_pad"])),
        int(abs(face_row["y2"] - face_row["y2_pad"]))
    ]
    
    #print("padding:", padding)
    
    new_dim = padded_dim[0] - padding[0] - padding[2], padded_dim[1] - padding[1] - padding[3]
    diff_x, diff_y = (target_dim[0] - new_dim[0]) / 2, (target_dim[1] - new_dim[1]) / 2
    #print("second diff:", diff_x, diff_y)
    
    padding[0] -= int(np.floor(diff_x))
    padding[1] -= int(np.floor(diff_y))
    padding[2] -= int(np.ceil(diff_x))
    padding[3] -= int(np.ceil(diff_y))
    
    #print("mask size after:", mask.shape)
    #print("unpadded bbox:", (face_row["x1"], face_row["y1"], face_row["x2"], face_row["y2"]))
    #print("adjusted padding:", padding)
    mask = mask[..., padding[1]:-padding[3], padding[0]:-padding[2]]
    
    return mask, True

In [ ]:
def pipeline(model, device):
    
    torch.autograd.set_detect_anomaly(True)
    
    df = pd.DataFrame() # dataframe storing the dataset
    row = {} #the information/columns for a single row in the dataset is stored here
    
    # Loop over all examples in test set
    for path in glob.glob(os.path.join(INPUT_PATH, '*.jpg')):
        row['path'] = path
        file_basename = os.path.basename(path)
        print(file_basename)
        
        model.eval()
            
        model.gradient_mode = False
        for yolo_layer in model.yolo_layers:
            yolo_layer.gradient_mode = False
        
        # read and transform the image from the path
        data = cv2.imread(path)  # read the image
        data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB) #change to rgb
        data = transforms.Compose([DEFAULT_TRANSFORMS,Resize(416)])((data, np.zeros((1, 5))))[0].unsqueeze(0) # transform the image
        
        # Forward pass the data through the model and call non max suppression
        nms, nms_output = non_max_suppression(model(data), 0.5, 0.5) #conf_thres and iou_thres = 0.5
        
        face_list = []
        if type(nms_output[0]) is not int:
            face_list = nms_output[0]
        
        data = data.to(device)
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True
        
        model.gradient_mode = True
        for yolo_layer in model.yolo_layers:
            yolo_layer.gradient_mode = True
        
        output = model(data)
        
        # loop through each of the faces in the image
        for face_index, face_row in enumerate(face_list): #nms_output[0] because the model is designed to take in several images at a time from the dataloader but we are only loading the image one at a time
            row['face_index'] = face_index
            print("Face", face_index)
            
            x, y, w, h = face_row[0], face_row[1], face_row[2], face_row[3]
            
            factor_x, factor_y, factor_w, factor_h = random.uniform(1, 2), random.uniform(1, 2), random.uniform(1, 2), random.uniform(1, 2)
            normal_x, normal_y, normal_w, normal_h = x / 416, y / 416, w / 416, h / 416
            
            new_x = normal_x * factor_x if random.choice([True, False]) else normal_x / factor_x
            new_y = normal_y * factor_y if random.choice([True, False]) else normal_y / factor_y
            new_w = normal_w * factor_w if random.choice([True, False]) else normal_w / factor_w
            new_h = normal_h * factor_h if random.choice([True, False]) else normal_h / factor_h
            
            new_x, new_y, new_w, new_h = max(min(1, new_x), 0), max(min(1, new_y), 0), max(min(1, new_w), 0), max(min(1, new_h), 0)
            
            target = torch.tensor([[0.0, 0, new_x, new_y, new_w, new_h]])
            target = target.to(device)
            
            loss, loss_components = compute_loss(output, target, model)
            
            # cropped image with bounding box
            # getting (x1, y1) upper left, (x2, y2) lower right
            x1 = max(int(np.floor((x - w / 2).detach().cpu().numpy())), 0)
            y1 = max(int(np.floor((y - h / 2).detach().cpu().numpy())), 0)
            x2 = min(int(np.ceil((x + w / 2).detach().cpu().numpy())), 415)
            y2 = min(int(np.ceil((y + h / 2).detach().cpu().numpy())), 415)
            
            row['x1'], row['y1'], row['x2'], row['y2'] = x1, y1, x2, y2
            
            cropped_image = detach_cpu(data)[:, :, y1:y2, x1:x2] #get the first dimension, the channels, and crop it
            cropped_image = tensor_to_image(cropped_image) #reshape the image to (w/h, h/w, channel)
            
            # Zero all existing gradients
            model.zero_grad()
            data.grad = None

            # Calculate gradients of model in backward pass
            loss.backward(retain_graph=True) #TODO: Amos - check if this is correct
            
            # Collect datagrad
            data_grad = data.grad.data
            #print('Gradient')
            #print(data_grad)
            #print(data_grad.shape)
            #plt.imshow(np.transpose(np.clip(data_grad.squeeze(0).numpy(), 0, 1), (1, 2, 0)))
            #plt.show()
            
            bbox = (x1, y1, x2, y2)
            mask, used_mask = load_mask(os.path.basename(path), face_index, bbox)
            row['used_mask'] = used_mask
            inverted_mask = 1 - mask
            
            #TODO: Jay - extract image attributes here
            # extract the image attributes from  the 'cropped_image' variable
            # save the attributes as row['<column name in the dataset>'] = <data> (see examples above for reference)
            
            row = extract_image_attributes(row, path, face_index, cropped_image * tensor_to_image(mask[0]))
            
            #print("bbox dim:", bbox)
            #print(mask.shape, data[:, :, y1:y2, x1:x2].shape)
            # TODO - Amos - determine the value of epsilon by calling fgsm_attack and changing the value of epsilon (see code below)
            # the value of data(image) and data_grad remains constant diba
            
            #print("Calculating min epsilon for YuNet...")
            yn_min_e_face = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yn_det_fn, mask, bbox)
            #print("Calculating min epsilon for MediaPipe...")
            mp_min_e_face = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.mp_det_fn, mask, bbox)
            #print("Calculating min epsilon for YoloFace...")
            yf_min_e_face = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yf_det_fn, mask, bbox)
            print("yunet min face:", yn_min_e_face, "mediapipe min face:", mp_min_e_face, "yoloface min face:", yf_min_e_face)
            
            row['e_face_yn'], row['e_face_mp'], row['e_face_yf'] = yn_min_e_face, mp_min_e_face, yf_min_e_face
            
            if used_mask:
                yn_min_e_bg = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yn_det_fn, inverted_mask, bbox)
                mp_min_e_bg = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.mp_det_fn, inverted_mask, bbox)
                yf_min_e_bg = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yf_det_fn, inverted_mask, bbox)
            else:
                yn_min_e_bg = yn_min_e_face
                mp_min_e_bg = mp_min_e_face
                yf_min_e_bg = yf_min_e_face
                
            print("yunet min bg:", yn_min_e_bg, "mediapipe min bg:", mp_min_e_bg, "yoloface min bg:", yf_min_e_bg)
            
            row['e_bg_yn'], row['e_bg_mp'], row['e_bg_yf'] = yn_min_e_bg, mp_min_e_bg, yf_min_e_bg
            
            cur_filename = os.path.join(PERTURBED_OUTS, str(face_index) + "yn_min_e_face_" + file_basename)
            save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), yn_min_e_face, data_grad.clone().detach(), mask, *bbox), cur_filename)
            cur_filename = os.path.join(PERTURBED_OUTS, str(face_index) + "mp_min_e_face_" + file_basename)
            save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), mp_min_e_face, data_grad.clone().detach(), mask, *bbox), cur_filename)
            cur_filename = os.path.join(PERTURBED_OUTS, str(face_index) + "yf_min_e_face_" + file_basename)
            save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), yf_min_e_face, data_grad.clone().detach(), mask, *bbox), cur_filename)
            cur_filename = os.path.join(PERTURBED_OUTS, str(face_index) + "yn_min_e_bg_" + file_basename)
            save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), yn_min_e_bg, data_grad.clone().detach(), mask, *bbox), cur_filename)
            cur_filename = os.path.join(PERTURBED_OUTS, str(face_index) + "mp_min_e_bg_" + file_basename)
            save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), mp_min_e_bg, data_grad.clone().detach(), mask, *bbox), cur_filename)
            cur_filename = os.path.join(PERTURBED_OUTS, str(face_index) + "yf_min_e_bg_" + file_basename)
            save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), yf_min_e_bg, data_grad.clone().detach(), mask, *bbox), cur_filename)
            
            # Call FGSM Attack
            
            #perturbed_data = minE.fgsm_attack(data.clone().detach(), yf_min_e, data_grad.clone().detach(), mask, *bbox)
            #perturbed_data = fgsm_attack(data, max(yn_min_e, mp_min_e), data_grad) #data is the input image, epsilon
            #print("can detect faces on unperturbed img?", minE.mp_det_fn(data.detach()))
            #print(f"can detect faces on perturbed data with e={max(yn_min_e, mp_min_e) - 0.01}?", minE.mp_det_fn(fgsm_attack(data, max(yn_min_e, mp_min_e) - 0.01, data_grad).detach()))
            #print(f"can detect faces on perturbed img? with e={max(yn_min_e, mp_min_e) - 0.01}", minE.mp_det_fn(perturbed_data.detach()))
            
            df = df.append(row, ignore_index=True) #append the attributes of one face to the dataframe
            
#             plt.imshow(tensor_to_image(perturbed_data))
            plt.show()
            
    df.to_csv(os.path.join(CSV_PATH, FOLDER_NAME + '_features_dataset' + str(int(time.time())) + '.csv'), index=False)  #save to csv

In [ ]:
folders = ["test"]
OUTPUT_FOLDER = os.path.join(os.getcwd(), "WIDER")
FOLDER_NAME = ""

for FOLDER_NAME in folders:
    INPUT_PATH = os.path.join(os.getcwd(), 'images', FOLDER_NAME)
    FOLDER_PATH = os.path.join(OUTPUT_FOLDER, FOLDER_NAME)
    CSV_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_CSV')
    RESTORED_MASK_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_restored_mask')
    PERTURBED_OUTS = os.path.join(FOLDER_PATH, FOLDER_NAME + "_perturbed_outs")
    
    if not os.path.exists(PERTURBED_OUTS):
        os.mkdir(PERTURBED_OUTS)
    
    CSV_FILE = ""
    for file in os.listdir(CSV_PATH):
        if "dataset_pixels" in file and file.endswith(".csv"):
            CSV_FILE = os.path.join(os.getcwd(), CSV_PATH, file)
    #print(CSV_FILE)
    #raise ex
    print("Working on", FOLDER_NAME, "folder")
    pipeline(model, device)

In [ ]:
# from torchview import draw_graph
# path = 'E:\\Documents\\GitHub\\THS-ST1\\images\\Test\\girl.jpg'

device, model = load_model('./weights/yolo_face_sthanhng.cfg', "./weights/yolo_face_sthanhng.weights")

# model.eval()
# model.gradient_mode = True
# for yolo_layer in model.yolo_layers:
#     yolo_layer.gradient_mode = True

# # model_graph = draw_graph(model, input_size=(1, 3, 416, 416), device='meta', save_graph=True, mode='train', filename='train_graph')
# # model_graph.visual_graph
# data = cv2.imread(path)  # read the image
# # print(data)
# data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB) #change to rgb
# data = transforms.Compose([DEFAULT_TRANSFORMS,Resize(416)])((data, np.zeros((1, 5))))[0].unsqueeze(0) # transform the image
# output1 = model(data)
# output2 = model(data)
# target = torch.tensor([[0.0, 0.0, 0.5, 0.5, 0.5, 0.5]])
# target = target.to(device)
# print(target.shape)
# output = model(data)
# for x in output:
#     print(x.shape)

# model.gradient_mode = False
# for yolo_layer in model.yolo_layers:
#     yolo_layer.gradient_mode = False

# output = model(data)
# print(output.shape)
# # compute_loss(output, target, model)
# # print(output.shape)

# # for i, yolo_layer in enumerate(model.yolo_layers):
# #     yolo_layer.gradient_mode = True
# #     print(yolo_layer.gradient_mode)

___